# FixedIncomeLib SABR Model Testing Notebook
This notebook exercises the `SabrModel` class by:
1. Building a dummy SABR volatility surface for a single index.
2. Interpolating volatilities at various expiry/tenor grid points.
---

In [21]:
import sys, os

# compute the parent of tests → FixedIncomeLib
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print("Added to sys.path:", repo_root)

Added to sys.path: c:\Users\neels\OneDrive\Desktop\Capstone_Project\FixedIncomeLib


In [22]:
import pandas as pd
from sabr import SabrModel
from yield_curve import YieldCurve

2. Dummy Yield Curve for subModel (flat rates)
We build a simple flat curve to satisfy SabrModel's subModel requirement.

In [23]:
yc_data = pd.DataFrame([
    ['SOFR-1B', '1M', 0.0020],
    ['SOFR-1B', '3M', 0.0025],
    ['USD-LIBOR-BBA-3M', '1M', 0.0020],
    ['USD-LIBOR-BBA-3M', '3M', 0.0025],
], columns=['INDEX','AXIS1','VALUES'])
yc_build = [
    {'TARGET': 'SOFR-1B',          'INTERPOLATION METHOD': 'PIECEWISE_CONSTANT'},
    {'TARGET': 'USD-LIBOR-BBA-3M', 'INTERPOLATION METHOD': 'PIECEWISE_CONSTANT'},
]

valueDate = "2025-06-26"
yc = YieldCurve(valueDate, yc_data, yc_build)
print("YieldCurve built successfully.")

YieldCurve built successfully.


3. Dummy SABR Data
Columns: INDEX, EXPIRY (years), TENOR (years), NORMALVOL, BETA, NU, RHO

In [24]:
sabr_data = pd.DataFrame([
    ['USD-LIBOR-BBA-3M', 1.0, 0.25, 0.010, 0.5, 0.2, -0.3],
    ['USD-LIBOR-BBA-3M', 1.0, 0.50, 0.011, 0.5, 0.2, -0.3],
    ['USD-LIBOR-BBA-3M', 2.0, 0.25, 0.012, 0.5, 0.2, -0.3],
    ['USD-LIBOR-BBA-3M', 2.0, 0.50, 0.013, 0.5, 0.2, -0.3],
], columns=['INDEX','AXIS1','AXIS2','NORMALVOL','BETA','NU','RHO'])

In [25]:
build_methods = [
    {
        'TARGET':       'USD-LIBOR-BBA-3M',
        'AXIS1':        'AXIS1',
        'AXIS2':        'AXIS2',
        'VALUES':       'NORMALVOL',
        'INTERPOLATION':'LINEAR',
    },
    {
        'TARGET':       'USD-LIBOR-BBA-3M',
        'AXIS1':        'AXIS1',
        'AXIS2':        'AXIS2',
        'VALUES':       'BETA',
        'INTERPOLATION':'LINEAR',
    },
    {
        'TARGET':       'USD-LIBOR-BBA-3M',
        'AXIS1':        'AXIS1',
        'AXIS2':        'AXIS2',
        'VALUES':       'NU',
        'INTERPOLATION':'LINEAR',
    },
    {
        'TARGET':       'USD-LIBOR-BBA-3M',
        'AXIS1':        'AXIS1',
        'AXIS2':        'AXIS2',
        'VALUES':       'RHO',
        'INTERPOLATION':'LINEAR',
    },
]
print("Build methods for all SABR params set.")


Build methods for all SABR params set.


In [26]:
sabr = SabrModel(valueDate, sabr_data, build_methods, yc, shift=0.0)
print("SabrModel initialized and calibrated.")

SabrModel initialized and calibrated.


In [27]:
print("Registered SABR components:", list(sabr.components.keys()))

Registered SABR components: ['USD-LIBOR-BBA-3M-NORMALVOL', 'USD-LIBOR-BBA-3M-BETA', 'USD-LIBOR-BBA-3M-NU', 'USD-LIBOR-BBA-3M-RHO']


In [28]:
grid = [(1.0, 0.25), (1.0, 0.50), (2.0, 0.25), (2.0, 0.50)]
print("--- SABR Volatility Interpolation ---")
for expiry, tenor in grid:
    normalvol, beta, nu, rho = sabr.get_sabr_parameters(
        "USD-LIBOR-BBA-3M",
        expiry,
        tenor
    )
    print(f"Expiry={expiry}y, Tenor={tenor}y → Vol = {normalvol:.4%}")


--- SABR Volatility Interpolation ---
Expiry=1.0y, Tenor=0.25y → Vol = 1.0000%
Expiry=1.0y, Tenor=0.5y → Vol = 1.1000%
Expiry=2.0y, Tenor=0.25y → Vol = 1.2000%
Expiry=2.0y, Tenor=0.5y → Vol = 1.3000%


In [29]:
print("--- Off-Grid SABR Volatility Interpolation ---")
off_grid = [
    (1.5, 0.25),
    (1.0, 0.375),
    (1.5, 0.375)
]
target = "USD-LIBOR-BBA-3M"
for expiry, tenor in off_grid:
    normalvol, beta, nu, rho = sabr.get_sabr_parameters(target, expiry, tenor)
    print(f"Expiry={expiry:>4.3f}y, Tenor={tenor:>5.3f}y → Vol = {normalvol:.4%}")

--- Off-Grid SABR Volatility Interpolation ---
Expiry=1.500y, Tenor=0.250y → Vol = 1.1000%
Expiry=1.000y, Tenor=0.375y → Vol = 1.0500%
Expiry=1.500y, Tenor=0.375y → Vol = 1.1500%


In [30]:
print("=== SABR Model via from_curve (prebuilt YieldCurve) ===")
sabr_a = SabrModel.from_curve(
    valueDate,
    sabr_data,
    build_methods,
    yc,          # prebuilt YieldCurve
    shift=0.0
)
for expiry, tenor in [(1.5,0.375), (2.0,0.5)]:
    vol = sabr_a.get_sabr_parameters("USD-LIBOR-BBA-3M", expiry, tenor)[0]
    print(f"[from_curve] Expiry={expiry}, Tenor={tenor} → Vol={vol:.4%}")

print("\n=== SABR Model via from_data (raw YC data & build methods) ===")
sabr_b = SabrModel.from_data(
    valueDate,
    sabr_data,
    build_methods,
    yc_data,
    yc_build,
    shift=0.0
)
for expiry, tenor in [(1.5,0.375), (2.0,0.5)]:
    vol = sabr_b.get_sabr_parameters("USD-LIBOR-BBA-3M", expiry, tenor)[0]
    print(f"[from_data ] Expiry={expiry}, Tenor={tenor} → Vol={vol:.4%}")


=== SABR Model via from_curve (prebuilt YieldCurve) ===
[from_curve] Expiry=1.5, Tenor=0.375 → Vol=1.1500%
[from_curve] Expiry=2.0, Tenor=0.5 → Vol=1.3000%

=== SABR Model via from_data (raw YC data & build methods) ===
[from_data ] Expiry=1.5, Tenor=0.375 → Vol=1.1500%
[from_data ] Expiry=2.0, Tenor=0.5 → Vol=1.3000%
